# Homework Assignment #4 (Individual)
## Regression and SVM Predictive Models

### <p style="text-align: right;"> &#9989; TJ McKenna</p>
### <p style="text-align: right;"> &#9989; tjmcken22</p>

### Goal for this homework assignment
By now, you have learned a bit about predictive models. In this assignment, you will practice:

* Building predictive models using `statsmodels` and `sklearn`
* Evaluating your predictive models

**This assignment is due roughly two weeks from now at 11:59 pm on Friday, April 21st.** It should be uploaded into the "Homework Assignments" submission folder for Homework #4.  Submission instructions can be found at the end of the notebook.


---
## Part 0: Add to your Git repository to track your progress on your assignment (4 points)

For this assignment, you're going to add it to the `cmse202-s23-turnin` repository you created in class so that you can track your progress on the assignment and preserve the final version that you turn in. In order to do this you need to

**&#9989; Do the following**:

1. Navigate to your `cmse202-s23-turnin` repository and create a new directory called `hw-04`.
2. Move this notebook into that **new directory** in your repository, then **add it and commit it to your repository**.
1. Finally, to test that everything is working, "git push" the file so that it ends up in your GitHub repository.

**Important**: Double check you've added your Professor and your TA as collaborators to your "turnin" repository (you should have done this in the previous homework assignment).

**Also important**: Make sure that the version of this notebook that you are working on is the same one that you just added to your repository! If you are working on a different copy of the notebook, **none of your changes will be tracked**!

If everything went as intended, the file should now show up on your GitHub account in the "`cmse202-s23-turnin`" repository inside the `hw-04` directory that you just created.  Periodically, **you'll be asked to commit your changes to the repository and push them to the remote GitHub location**. Of course, you can always commit your changes more often than that, if you wish.  It can be good to get into a habit of committing your changes any time you make a significant modification, or when you stop working on the project for a bit.

&#9989; **Do this**: Before you move on, put the command that your instructor should run to clone your repository in the markdown cell below.

Git clone 'url' 

# Building a Model to Predict if it Will Rain Tomorrow

In this dataset we have weather data for a number of Australian cities. We’re going to **build a model that uses data from today to predict if it will rain tomorrow.** To simplify our model, let’s just look at the weather for one city.

## Part 1. Working with the Data

The dataset that we’ll be using can be found here:
- `https://raw.githubusercontent.com/msu-cmse-courses/cmse202-S23-data/main/HW/HW4/aussie_weather_data.csv`

#### 1.1 (3 Points)
**Download the data set and read it in using Pandas.**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.datasets import fetch_lfw_people
import time
from sklearn.datasets import make_circles
from mpl_toolkits.mplot3d import Axes3D

In [3]:
#Write your code here
import pandas as pd
import numpy as np

df = pd.read_csv('aussie_weather_data.csv', delimiter = ',')
df.head()

,Year,Month,DayOfMonth,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2009,1,1,NorfolkIsland,20.4,25.8,0.0,6.0,12.4,112.5,...,84.0,71.0,1014.5,1013.6,3.0,1.0,23.3,24.7,0,0
1,2009,1,2,NorfolkIsland,20.9,26.7,0.2,8.0,10.3,90.0,...,79.0,77.0,1016.3,1015.5,2.0,5.0,25.0,25.1,0,0
2,2009,1,3,NorfolkIsland,22.3,26.3,0.0,3.2,2.0,0.0,...,87.0,90.0,1014.6,1014.3,7.0,7.0,24.7,23.8,0,1
3,2009,1,4,NorfolkIsland,21.6,22.2,1.2,2.8,0.0,337.5,...,92.0,95.0,1016.0,1015.3,8.0,8.0,22.1,21.2,1,1
4,2009,1,5,NorfolkIsland,20.4,23.5,2.6,2.2,2.9,337.5,...,86.0,86.0,1015.3,1013.7,7.0,7.0,21.8,21.6,1,0


#### 1.2 (3 Points)
This dataset has weather information for multiple cities in Australia. It’s quite a large dataset. In fact, it’s a little *too* large for our purposes; it will take a considerable amount of time to train a model on so much data. So we’ll just work with the data for a specific city.

The first thing we’ll do is see which cities we have data for. 
**Print out a list of the specific cities in this dataset.** (Note that this is not the same as printing the entire `Location` column.)

In [4]:
#Write your code here
df.Location.unique()

array(['NorfolkIsland', 'SydneyAirport', 'WaggaWagga', 'MelbourneAirport',
       'Mildura', 'Watsonia', 'Brisbane', 'Cairns', 'Townsville',
       'MountGambier', 'Nuriootpa', 'PerthAirport', 'Perth',
       'AliceSprings', 'Darwin'], dtype=object)

#### 1.3 (3 Points)
Select one of the cities (and **please** don't just pick the first one!). Cut down your dataframe so that it only contains data from your chosen city. You should also **look through the columns in this dataset and ensure that all of the data is ready for our analysis/model creation.**

In [5]:
#Write your code here
brisbane = df[df['Location'] == 'Brisbane']
brisbane

,Year,Month,DayOfMonth,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
16003,2008,7,1,Brisbane,10.3,25.2,0.0,1.4,9.5,157.5,...,81.0,37.0,1019.6,1014.8,0.0,1.0,14.9,24.6,0,0
16004,2008,7,2,Brisbane,8.1,22.9,0.0,2.0,9.8,180.0,...,41.0,30.0,1018.8,1015.0,0.0,0.0,16.2,22.4,0,0
16005,2008,7,3,Brisbane,9.7,22.4,0.0,5.8,9.4,0.0,...,55.0,52.0,1021.4,1019.1,1.0,4.0,15.4,21.3,0,0
16006,2008,7,4,Brisbane,11.8,20.0,0.8,1.8,1.1,225.0,...,76.0,53.0,1023.5,1021.7,7.0,7.0,14.1,19.6,0,0
16007,2008,7,5,Brisbane,12.3,16.7,0.0,2.0,0.3,270.0,...,81.0,89.0,1027.3,1026.2,7.0,8.0,16.1,15.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18951,2017,6,21,Brisbane,12.2,23.4,0.0,4.0,9.2,202.5,...,61.0,44.0,1023.4,1021.3,1.0,4.0,17.2,23.3,0,0
18952,2017,6,22,Brisbane,13.7,22.7,0.0,3.4,7.2,315.0,...,62.0,55.0,1026.0,1022.4,2.0,3.0,17.8,20.5,0,0
18953,2017,6,23,Brisbane,10.3,23.7,0.0,3.2,9.4,180.0,...,71.0,37.0,1023.3,1018.4,1.0,3.0,15.2,23.1,0,0
18954,2017,6,24,Brisbane,10.4,24.5,0.0,3.4,8.9,270.0,...,75.0,33.0,1018.6,1015.4,7.0,5.0,14.3,24.0,0,0


#### 1.4 (3 Points)
Finally, before we start creating our models, let's first split our data into training and testing datasets. **Keep in mind what we want our model to predict and how this dataset gives you the information you need for your model.**

In [6]:
#Write your code here



x = brisbane.drop(columns = 'Location')
x = x.drop(columns = 'RainTomorrow')
y = brisbane['RainTomorrow']

train_vectors, test_vectors, train_labels, test_labels = train_test_split(x, y, train_size=.75, random_state=100, shuffle=True, stratify=None)


## 2.0 Building a Regression Model

#### 2.1 (3 Points)
We’ll start by creating a regression model. **Does it make sense for us to use linear regression or logistic regression? Explain your choice.**

Logistic regression is the better choice here as we want to find the most statistically significant parameters. This model is the best one to use for this situation.

#### 2.2 (12 Points)
Create a regression model. Use the model summary to determine which parameters are most statistically significant in your model. If you get an error during at this stage it may be helpful to go back to part 1.3 and ensure that all of your data is appropriate for creating a model. 

In [7]:
#Write your code here
logit_model = sm.Logit(train_labels, train_vectors)
result = logit_model.fit()
print(result.summary() )

Optimization terminated successfully.
         Current function value: 0.316753
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:           RainTomorrow   No. Observations:                 2214
Model:                          Logit   Df Residuals:                     2191
Method:                           MLE   Df Model:                           22
Date:                Thu, 20 Apr 2023   Pseudo R-squ.:                  0.3873
Time:                        21:22:13   Log-Likelihood:                -701.29
converged:                       True   LL-Null:                       -1144.6
Covariance Type:            nonrobust   LLR p-value:                2.533e-173
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Year             -0.0088      0.009     -1.011      0.312      -0.026       0.008
Month            -0.

Sunshine, WindGustSpeed, Humidity3pm, Pressure9am, Pressure3pm, Cloud3pm

#### 2.3 (8 Points)
**Fit your test data and create/print a confusion matrix. We’ll use this to evaluate how well your predicitve model performs.**

In [8]:
#Write your code here
predicted_vals = result.predict(test_vectors)
pred_labels = list(map(round, predicted_vals))

print(confusion_matrix(test_labels, pred_labels))

[[539  28]
 [ 73  99]]


#### 2.4 (3 Points)
Look at/calculate the Precision and Recall of your model. **Which one is greater? Describe in plain language what that means about the performance of your model (I.e., the circumstances in which it does/doesn’t do well).**

In [9]:
from sklearn.metrics import precision_score, recall_score
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(train_vectors, train_labels)

y_pred = classifier.predict(test_vectors)
print(precision_score(test_labels,y_pred, average="macro"))
print(recall_score(test_labels,y_pred, average="macro"))

0.820615253975606
0.7555217177310201


/home/mcken248/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [10]:
start = time.time()

#make some temporary variables so you can change this easily
tmp_vectors = train_vectors
tmp_labels = train_labels

print("Fitting the classifier to the training set")
# a dictionary of hyperparameters: key is the name of the parameter, value is a list of values to test
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
              'kernel': ['logistic','rbf']}
# make a classifier by searching over a classifier and the parameter grid
clf = GridSearchCV(SVC(class_weight='balanced'), param_grid)

# we have a "good" classifier (according to GridSearchCV), how's it look
clf = clf.fit(tmp_vectors, tmp_labels)
print("Best estimator found by grid search:")
print(clf.best_estimator_)
print("Best parameters found by grid search:")
print(clf.best_params_)

end = time.time()
print("Runtime",end - start)

Fitting the classifier to the training set


/home/mcken248/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
150 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
150 fits failed with the following error:
Traceback (most recent call last):
  File "/home/mcken248/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/mcken248/.local/lib/python3.8/site-packages/sklearn/svm/_base.py", line 180, in fit
    self._validate_params()
  File "/home/mcken248/.local/lib/python3.8/site-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/home/mcken248/.l

Best estimator found by grid search:
SVC(C=1000.0, class_weight='balanced', gamma=0.0005)
Best parameters found by grid search:
{'C': 1000.0, 'gamma': 0.0005, 'kernel': 'rbf'}
Runtime 67.29746198654175


In [11]:
predict_vectors = test_vectors
true_labels = test_labels

print("Predicting names on the test set")
pred_labels = clf.predict(predict_vectors)

print(classification_report(true_labels, pred_labels))
print(confusion_matrix(true_labels, pred_labels))

Predicting names on the test set
              precision    recall  f1-score   support

           0       0.85      0.87      0.86       567
           1       0.54      0.51      0.53       172

    accuracy                           0.79       739
   macro avg       0.70      0.69      0.69       739
weighted avg       0.78      0.79      0.78       739

[[493  74]
 [ 84  88]]


The precision score is higher which means the data is returning mostly relevant results, meaning it tests well with the trained data.

## 3.0 Interlude: A Base Rate Frequency Model (12 Points)

If we didn't create a fancy model for predicting future rainfall, would that mean that we had *no way* of predicting if it would rain? Of course not. There's always *some* kind of model we can use (they just won't be very sophisticated). 

In the case of our problem--predicting whether it will rain tomorrow--what is the simplest model we could use? We'd need to look at the **base rate frequency**. If we know that, over the course of the year, it rains ~30% of the time, that would be our base rate frequency. If we were to guess that it would be sunny every day of the year, we'd be wrong ~30% of the time, *but we'd be right ~70% of the time.* 

Why is this important? Imagine we create a sophisticated model that accurately predicts rain/no rain ~60% of the time; without any other information, we might think, "That's not too bad, I guess." It certainly feels better than *no* information, right? Now think about the base rate frequency model, which achieves an accuracy of **70%** just by guessing the same thing every time. Suddenly, our sophisticated model seems like crap! It tells us we could be doing ~10% better than our fancy model by guessing. This is why we use the base rate frequency as a simple way to test our models.


**Calculate how frequently it rains in your city. If you were to guess that it *wasn't* going to rain every day, how frequently would you be right? (We'll use this as our baseline model.)**


In [12]:
#Write your code for calculating the frequency of rain here.
rain_today = brisbane['RainToday']

print(rain_today.value_counts())
print(len(rain_today))
print('rain_freq:', 642/len(rain_today)*100, '%')
print('correct_freq:', 2311/len(rain_today)*100, '%')

0    2311
1     642
Name: RainToday, dtype: int64
2953
rain_freq: 21.740602776837115 %
correct_freq: 78.25939722316289 %


About 78% of the time

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

## 4.0 SVM Model

We’ve seen how regression faired in predicting rain; now let’s give Support Vector Machines a shot. 

It’s not clear, *a priori*, which type of kernel (linear or RBF) would fair better. Therefore, we’ll need to test out both kernel types, as well as multiple values for hyperparameters ($C$ and $\gamma$). 

#### 4.1 (12 Points)
Run a grid search over both types of kernels and multiple values of $C$ and $\gamma$ (we recommend [0.01,0.1,1.0,10.0] for both). We *STRONGLY* recommend you set the parameter `n_jobs=-1` in `GridSearchCV`, which will distribute the computational load. Make sure to write down your best-fit hyperparameters. 

In [13]:
#Write your code here
linear = GridSearchCV(SVC(kernel = 'linear', class_weight = 'balanced'), param_grid, n_jobs = -1)
clf_linear = linear.fit(train_vectors, train_labels)

print("Best estimator found by grid search:")
print(clf_linear.best_estimator_)
print("Best parameters found by grid search:")
print(clf_linear.best_params_)

/home/mcken248/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
150 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/home/mcken248/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/mcken248/.local/lib/python3.8/site-packages/sklearn/svm/_base.py", line 180, in fit
    self._validate_params()
  File "/home/mcken248/.local/lib/python3.8/site-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/home/mcken248/.loc

Best estimator found by grid search:
SVC(C=1000.0, class_weight='balanced', gamma=0.0005)
Best parameters found by grid search:
{'C': 1000.0, 'gamma': 0.0005, 'kernel': 'rbf'}


gamma = .0001

#### 4.2 (8 Points)
Using your best-fit parameters, classify your test data and print off the confusion matrix. Note/calculate the Precision and Recall for your SVM model, as well as the overall accuracy.

$\mathrm{Accuracy} = \frac{\mathrm{Accurate~Predictions}}{\mathrm{Total~Samples}}$.

In [14]:
#Write your code here
predict_vectors = test_vectors
true_labels = test_labels

print("Predicting names on the test set")
pred_labels = clf_linear.predict(predict_vectors)

print(classification_report(true_labels, pred_labels))
print(confusion_matrix(true_labels, pred_labels))

Predicting names on the test set
              precision    recall  f1-score   support

           0       0.85      0.87      0.86       567
           1       0.54      0.51      0.53       172

    accuracy                           0.79       739
   macro avg       0.70      0.69      0.69       739
weighted avg       0.78      0.79      0.78       739

[[493  74]
 [ 84  88]]


In [15]:
Acc = (93+495)/(495+67+84+93)
print(Acc*100, '% accurate')

79.56698240866035 % accurate


In [16]:
y_pred = clf_linear.predict(test_vectors)
print(precision_score(test_labels,y_pred, average="macro"))
print(recall_score(test_labels,y_pred, average="macro"))

0.698814643644222
0.6905582215659736


## 5. Evaluation of Models

Here we’re going to investigate how well our predictive models actually work.


#### 5.1 (5 Points)

Compare the accuracy of both of your predictive models (regression and SVM) to the baseline model from part 3. Do your predictive models do better than the baseline model? Is their performance relative to the baseline model what *you* would expect? 


The performance is certainly relative to the baseline model as the numbers are quite similar, but I think the predictive models do a better job as it's a bit more accurate and actually runs a full test through train and test data to find our answer.

#### 5.2 (5 Points)

Let’s test the robustness of your model. Before you do anything, you should record the precision, recall, and accuracy of your two predictive models (if you haven’t already). Once you’ve done that, go all the way back to part 1.4 and change the random seed for `train_test_split`. Using this new data split, rerun all of your code from parts 2 and 4. For each new random seed, record the precision, recall, and accuracy for both models (and write them, as well as the random seed, in the cell below!). Do this for 3-5 different random seeds. 


1. Random State = 42
    -Regression: Recall = .76, Precision = .85, Acc = 79.5%
    -SVM: Recall = .72, Precision = .73, Acc = 79.5%
    
2. Random State = 2
    -Regression: Recall = .72, Precision = .77, Acc = 79.5%
    -SVM: Recall = .70, Precision = .69, Acc = 79.5%
    
3. Random State = 100
    -Regression: Recall = .75, Precision = .82, Acc = 79.5%
    -SVM: Recall = .69, Precision = .70, Acc = 79.5%

#### 5.3 (5 Points)

How much variation did you observe in the precision, recall, and accuracy in your models? After going through this exercise, how (if at all) does this change your response to part 5.1? 

The accuracy remained the same regardless of the random seed, but there were instances of the precision and recall decreasing, which would maybe make me reconsider. However, I still think the predictive modeling is better.

#### 5.4 (5 Points)

Finally, compare the performance of your regression and SVM models. Which, if any, performed better? Esxplain.

I think the regression model performed better as it consistantly had higher precision values than recall values, which tells me the data returns more relevant results than the SVM model.

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

### Congratulations, you're done!

&#169; Copyright 2022,  Department of Computational Mathematics, Science and Engineering at Michigan State University